In [1]:
%pip install llama-index-llms-openai
%pip install llama-index-readers-file
%pip install python-dotenv
%load_ext autoreload
%autoreload 2


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load environment variables from .env file
import pandas as pd
from llama_index.core.evaluation import (
    RetrieverEvaluator,
    get_retrieval_results_df,
)
import os
import logging
import pandas as pd
from dotenv import load_dotenv
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter
from datetime import datetime
from llama_index.core import (
    StorageContext, VectorStoreIndex, SimpleDirectoryReader, 
    get_response_synthesizer, Settings
)
import traceback
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
from llama_index.core.evaluation import generate_question_context_pairs, QueryResponseDataset
from llama_index.llms.openai import OpenAI

import nest_asyncio

nest_asyncio.apply()

from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
import json
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.embeddings import resolve_embed_model

######
import nest_asyncio
from llama_index.core import Settings

nest_asyncio.apply()
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)


load_dotenv()
# Fetch API keys from environment variables
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


llm = AzureOpenAI(
    deployment_name=AZURE_DEPLOYMENT_NAME,
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-large-en-v1.5"
)

Settings.llm = llm
Settings.embed_model=embed_model

def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]


/Users/rishi/Documents/MSc DSML/MSc Project/RAG Demo/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/rishi/Documents/MSc DSML/MSc Project/RAG Demo/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
# Loading docs
loader = PyMuPDFReader()
docs1 = loader.load(file_path="./LL144.pdf")
docs2 = loader.load(file_path="./LL144_Definitions.pdf")
docs = docs1 + docs2
#docs0 = loader.load_data(file=Path("policy.pdf"))
eval_dataset = EmbeddingQAFinetuneDataset.from_json("retriever_eval_dataset.json")

In [5]:
from llama_index.core.node_parser import SentenceWindowNodeParser

In [6]:
top_k=5
chunk_size=512
#chunk_overlap=chunk_size/2

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=5,
    window_metadata_key="window",
    original_text_metadata_key="original_sentence",
)
base_nodes = node_parser.get_nodes_from_documents(docs)
eval_dataset = generate_question_context_pairs(
    base_nodes, llm, num_questions_per_chunk=5
)

eval_dataset.save_json("sentence_window_eval_set.json")

# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

base_index = VectorStoreIndex(base_nodes, embed_model=embed_model)
base_retriever = base_index.as_retriever(similarity_top_k=top_k)

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"], retriever=base_retriever
)

results_base = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)

results_df = display_results(f"sentence_window", results_base)



  0%|          | 0/213 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 1065/1065 [01:04<00:00, 16.57it/s]


In [7]:
print(results_df)

        retrievers  hit_rate  mrr  precision  recall   ap  ndcg
0  sentence_window       0.0  0.0        0.0     0.0  0.0   0.0


In [9]:
results_base

[RetrievalEvalResult(query='What is the purpose of Subchapter 25 in relation to automated employment decision tools?', expected_ids=['75eea0d8-caae-43d9-ae64-a448215fc93e'], expected_texts=None, retrieved_ids=['node-176', 'node-1', 'node-0', 'node-68', 'node-2'], retrieved_texts=['Chapter 5 of title 20 of the administrative code of the city of New York is amended by adding\na new subchapter 25 to read as follows:\nSubchapter 25\nAutomated Employment Decision Tools\n§ 20-870 Definitions. ', 'For the purposes of this subchapter, the following terms have the following meanings:\n   Automated employment decision tool. ', 'Subchapter 25: Automated Employment Decision Tools\n§ 20-870 Definitions.\n', 'As used in this subchapter, the following terms have the following meanings: \n \nAutomated Employment Decision Tool. ', 'The term "automated employment decision tool" means any computational process, derived from machine\nlearning, statistical modeling, data analytics, or artificial intelligen

In [11]:
!pip install llama-index-vector-stores-qdrant

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached llama_index_vector_stores_qdrant-0.2.11-py3-none-any.whl.metadata (768 bytes)
  Using cached grpcio-1.64.1-cp310-cp310-macosx_12_0_universal2.whl.metadata (3.3 kB)
  Using cached protobuf-5.27.2-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached h2-4.1.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached hyperframe-6.0.1-py3-none-any.whl.metadata (2.7 kB)
  Using cached hpack-4.0.0-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_vector_stores_qdrant-0.2.11-py3-none-any.whl (10 kB)
Using cached grpcio-1.64.1-cp310-cp310-macosx_12_0_universal2.whl (10.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 18.0 MB/s eta 0:00:00a 0:00:01
Using cached h2-4.1.0-py3-none-any.whl (57 kB)
Using cached protobuf-5.27.2-cp38-abi3-macosx_10_9_universal2.whl (412 kB)
Using cached hpack-4.0.0-py3-none-any.whl (32 kB)
Using cached hyperframe-6.0.1-py3-none-any.whl (12 

In [18]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

def qdrant_hybrid_rag(docs,embed_model, alpha=0.5, similarity_top_k=2, sparse_top_k=2, chunk_size=512):
    embedding_model = embed_model
    #documents = load_pdf_data(document_file_path=['LL144.pdf', 'LL144_Definitions.pdf'], chunk_size=chunk_size)
    client = QdrantClient(path="./qdrant_data1")
    vector_store = QdrantVectorStore("ll144", client=client, enable_hybrid=True, batch_size=10)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    Settings.chunk_size = chunk_size
    splitter = SentenceSplitter(chunk_size=chunk_size)
    nodes = splitter.get_nodes_from_documents(docs)
    for idx, node in enumerate(base_nodes):
        node.id_ = f"node-{idx}"
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(docs, storage_context=storage_context, transformations=[splitter])
    query_engine = index.as_query_engine(alpha=alpha, similarity_top_k=similarity_top_k, sparse_top_k=sparse_top_k, vector_store_query_mode="hybrid")
    return query_engine, nodes, index

In [19]:
node_parser = SentenceSplitter(chunk_size=512)
base_nodes = node_parser.get_nodes_from_documents(docs)

query_engine, nodes, index = qdrant_hybrid_rag(docs=docs, embed_model=embed_model, alpha=0.5, similarity_top_k=5, sparse_top_k=5)

# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

hybrid_retriever = index.as_retriever(similarity_top_k=5)

ImportError: Could not import FastEmbed. Please install it with `pip install fastembed` or `pip install fastembed-gpu` for GPU support.
See `https://qdrant.github.io/fastembed/examples/FastEmbed_GPU/` for more details on GPU support.